In [ ]:
# TODO 
# 1. assign chen scores to khurana enhancers (in hg38 coordinates) (see aps.*) 
# 2. scatter plot zscores and Khurana scores 

In [ ]:
# TODO
# 1. pull out "vista" and "LoF tolerant" enhancers from "khurana_table_S2"
# 2. annotate all enhancers as "vista" or "LoF tolerant" or "neither"
# 3. Look at the CDF of scores (Chen, McHale, Khurana) for all enhancers, just those annotated as "vista", and just those annotated as "LoF-tolerant", similar to Fig 4A

In [ ]:
# TODO [LONG SHOT] 
# Do SNVs (het or hom alt) in vista enhancers avoid the motifs that are enriched in those enhancers? 